# Touches on the disc

Ultimate is a game of possession, similar to American football or European soccer. Similar to the latter, all players usually touch the disc while their team moves it around the field. And, similar to both sports, players tend to have skewed statistics on their usage percentage -- some players are high volume and others low.

Additionally, in the mixed division of the sport there is an asymmetry on the field. One gender will have 4 players and the other gender will have 3 (all on the same team, and the other team mirrors).

If both genders were on the field symmetrically, one would expect the usage percentage to be split between them (1:1 ratio) with all other factors being equal. Alternatively, if all positions were equivalent then one would expect the usage percent to be split according to a 4:3 ratio. Factoring in asymmetric positions, we turn to numerical simulations to find the expected usage ratios.

In [1]:
# First import a bunch of libraries
import numpy as np
import numpy.random as rand
import matplotlib.pyplot
%matplotlib inline

## Player object

Create a player object, to which we can assign it an ID number (0-6 for each player), gender, position, and a passing function. The passing function is an arbitrary function that takes in a gender, position, and team information and determines how the player will tend to throw to their teammates.

This player object+passing function scheme can work with any gender/position configuration. For instance on an open or womens team, the gender won't influence the passing ratio at all. Conversely, by letting the passing function be completely general, then one could design passing functions to test for systemic trends with a parametric model (this would be well beyond the scope of this notebook :-) ).

In [2]:
class Player(object):
    def __init__(self, ID, gender, position, passing_function):
        """Constructor used to create a player."""
        self.ID               = ID
        self.gender           = gender
        self.position         = position
        self.passing_function = passing_function
        if position not in ["h", "c"]:
            raise Exception("Currently only 'h' and 'c' positions supported.")

    def __str__(self):
        """Diagnostic for printing."""
        return "Player ID %d:\n\tGen: %s\n\tPos: %s\n\tPassing function: %s"%(self.ID,self.gender,self.position,self.passing_function.__name__)
        
    def make_pass(self, the_team):
        """A wrapper to the passing function, that takes in the team.
        Returns the ID number of a teammate that they have passed to."""
        return self.passing_function(self.ID, self.gender, self.position, the_team)

In [3]:
#Test that the player object is working
def fake_passing_function(ID, gender, position, the_team, *args):
    """Always pass to player 0."""
    return 0

Player1 = Player(1, "m", "h", fake_passing_function)
print str(Player1)
print "Pass result: ",Player1.make_pass(None)

Player ID 1:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Pass result:  0


## Team object

The team object is simple: it is a collection of players. This collection is used to run individual players' passing functions. It takes in an ordered list of genders, an ordered list of positions, and an ordered list of passing functions.

In [4]:
class Team(object):
    def __init__(self, gender_list, position_list, pf_list):
        """Constructor for the team."""
        if len(gender_list) != len(position_list) != len(pf_list):
            raise Exception("Must supply all information for all players.")
        N_players = len(gender_list)
        self.N_players = N_players
        if N_players < 2:
            raise Exception("Must have at least two players on the team.")
        self.players = [Player(i, gender_list[i], position_list[i], pf_list[i]) for i in range(N_players)]
        
    def __str__(self):
        out = str(self.players[0])
        for i in range(1,self.N_players):
            out += "\n"+str(self.players[i])
        return out

In [5]:
# Test that the team objects work.
gens = ["m" for i in range(7)]
poss = ["h","h","h","c","c","c","c"]
pfs = [fake_passing_function for i in range(7)]
Team1 = Team(gens, poss, pfs)
print Team1

Player ID 0:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 1:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 2:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 3:
	Gen: m
	Pos: c
	Passing function: fake_passing_function
Player ID 4:
	Gen: m
	Pos: c
	Passing function: fake_passing_function
Player ID 5:
	Gen: m
	Pos: c
	Passing function: fake_passing_function
Player ID 6:
	Gen: m
	Pos: c
	Passing function: fake_passing_function


## Usage object
Now we can make a usage object. This is an object that contains a team that has been initialized with a set of players. It simply simulates a lot of throws and records the information about the player that receives the disc.

In [6]:
class Usage(object):
    def __init__(self, the_team):
        self.the_team = the_team
        
    def simulate(self, N_throws, start_ID=0):
        print "Simulating with %d throws."%N_throws
        the_team = self.the_team
        print "Posession starting with:\n%s"%the_team.players[start_ID]
        allIDs  = [0]
        allgens = [the_team.players[start_ID].gender]
        allposs = [the_team.players[start_ID].position]
        current_ID = start_ID
        for i in range(N_throws):
            current_ID = the_team.players[current_ID].make_pass(the_team)
            allIDs.append(current_ID)
            allgens.append(the_team.players[current_ID].gender)
            allposs.append(the_team.players[current_ID].position)
        print "Simulation complete with %d throws."%N_throws
        return allIDs, allgens, allposs

In [7]:
# Test the Usage object
Simulation = Usage(Team1)
N_throws = 100
IDs, gens, poss = Simulation.simulate(N_throws)

Simulating with 100 throws.
Posession starting with:
Player ID 0:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Simulation complete with 100 throws.


## Passing functions
Now we need to create some passing functions. The most naive passing function will have a player randomly pass to any player besides themselves. Another passing function we could have is one in which a handler (position="h") always passes to a cutter (position="c").

In [8]:
def random_passing_function(ID, gender, position, the_team, *args):
    """Pure random passing function."""
    N = the_team.N_players
    IDs = range(0,ID)+range(ID+1,N)
    return rand.choice(IDs)

def hc_ch_passing_function(ID, gender, position, the_team, *args):
    """Handler-cutter cutter-handler passing function."""
    IDs = []
    for player in the_team.players:
        if position == "h":
            if player.position == 'c': IDs.append(player.ID)
        else:
            if player.position == 'h': IDs.append(player.ID)
    return rand.choice(IDs)

In [9]:
# Test the passing functions
receiver_IDs = [random_passing_function(0,'m','h',Team1) for i in range(20)]
print "IDs of random passes:\n\t",receiver_IDs

receiver_IDs = [hc_ch_passing_function(0,'m','h',Team1) for i in range(20)]
print "IDs of handler-cutter pass pairs:\n\t",receiver_IDs

IDs of random passes:
	[6, 4, 2, 2, 6, 5, 6, 4, 1, 3, 6, 1, 1, 6, 2, 1, 1, 6, 1, 3]
IDs of handler-cutter pass pairs:
	[6, 5, 3, 4, 6, 5, 6, 3, 4, 3, 5, 4, 3, 6, 5, 3, 3, 5, 3, 4]


## Experiment 1 - random team
The first experiment will be with a team entirely of players that pass randomly to other players. Let's do this with a 3-4 female-male ratio mixed team with two male and one female handlers, and we will simulate one million throws.

In [10]:
in_gens = ["m","f","m","f","m","f","m"]
in_poss = ["h","h","h","c","c","c","c"]
pfs = [random_passing_function for i in range(7)]
TeamExp1 = Team(in_gens, in_poss, pfs)
SimExp1 = Usage(TeamExp1)
N_throws = 1000000
IDs, gens, poss = SimExp1.simulate(N_throws)

Simulating with 1000000 throws.
Posession starting with:
Player ID 0:
	Gen: m
	Pos: h
	Passing function: random_passing_function
Simulation complete with 1000000 throws.


In [11]:
# Analysis on experiment one. 
# Basic statistic - the % of a male receiver vs. the % of a female receiver.
# We expect a 57% male vs 43% female ratio based on the 4:3 ratio of players.
N_possessions = N_throws+1 #One extra for the player starting with the disc.
N_male = gens.count("m")
N_female = gens.count("f")
P_male = float(N_male)/N_possessions*100
P_female = float(N_female)/N_possessions*100
print "Expected Percentages:\n\t43% f receivers\n\t57% m receivers"
print "Calculated Percentages for Experiment 1:\n\t%.2f%% f receivers\n\t%.2f%% m receivers"%(P_female, P_male)

Expected Percentages:
	43% f receivers
	57% m receivers
Calculated Percentages for Experiment 1:
	42.84% f receivers
	57.16% m receivers


## Experiment 2 - basic tactics team with 3:4 female-male ratio
The second experiment will be with a team entirely of players that pass randomly to players of the opposite position. So handlers always pass to cutters and cutters always to handlers. Let's do this with a 3-4 female-male ratio mixed team with two male and one female handlers, and we will simulate one million throws.

In [12]:
in_gens = ["m","f","m","f","m","f","m"]
in_poss = ["h","h","h","c","c","c","c"]
pfs = [hc_ch_passing_function for i in range(7)]
TeamExp2 = Team(in_gens, in_poss, pfs)
SimExp2 = Usage(TeamExp2)
N_throws = 1000000
IDs, gens, poss = SimExp2.simulate(N_throws)

Simulating with 1000000 throws.
Posession starting with:
Player ID 0:
	Gen: m
	Pos: h
	Passing function: hc_ch_passing_function
Simulation complete with 1000000 throws.


In [13]:
# Analysis on experiment two. 
# Basic statistic - the % of a male receiver vs. the % of a female receiver.
N_possessions = N_throws+1 #One extra for the player starting with the disc.
N_male = gens.count("m")
N_female = gens.count("f")
P_male = float(N_male)/N_possessions*100
P_female = float(N_female)/N_possessions*100
print "Expected Percentages of pure random throws:\n\t43% f receivers\n\t57% m receivers"
print "Calculated Percentages for Experiment 2:\n\t%.2f%% f receivers\n\t%.2f%% m receivers"%(P_female, P_male)

Expected Percentages of pure random throws:
	43% f receivers
	57% m receivers
Calculated Percentages for Experiment 2:
	41.71% f receivers
	58.29% m receivers


## Experiment 3 - handlers get 2/3 of passes
In the third experiment we can try a more complicated scenario in which any player, no matter what their position, will throw to a handler 66% of the time. This is somewhat realistic in a game setting, where tactics might include dump-swing-throw style play. To simulate this we need to construct a new passing function.

In [14]:
def exp3_passing_function(ID, gender, position, the_team, *args):
    hIDs = []
    cIDs = []
    for player in the_team.players:
        if player.position == 'c' and player.ID != ID: cIDs.append(player.ID)
        if player.position == 'h' and player.ID != ID: hIDs.append(player.ID)
    if rand.rand() < 0.66666:
        return rand.choice(hIDs)
    else:
        return rand.choice(cIDs)

In [15]:
in_gens = ["m","f","m","f","m","f","m"]
in_poss = ["h","h","h","c","c","c","c"]
pfs = [exp3_passing_function for i in range(7)]
TeamExp3 = Team(in_gens, in_poss, pfs)
SimExp3 = Usage(TeamExp3)
N_throws = 1000000
IDs, gens, poss = SimExp3.simulate(N_throws)

Simulating with 1000000 throws.
Posession starting with:
Player ID 0:
	Gen: m
	Pos: h
	Passing function: exp3_passing_function
Simulation complete with 1000000 throws.


In [16]:
# Analysis on experiment three. 
# Basic statistic - the % of a male receiver vs. the % of a female receiver.
N_possessions = N_throws+1 #One extra for the player starting with the disc.
N_male = gens.count("m")
N_female = gens.count("f")
P_male = float(N_male)/N_possessions*100
P_female = float(N_female)/N_possessions*100
print "Expected Percentages of pure random throws:\n\t43% f receivers\n\t57% m receivers"
print "Calculated Percentages for Experiment 3:\n\t%.2f%% f receivers\n\t%.2f%% m receivers"%(P_female, P_male)

Expected Percentages of pure random throws:
	43% f receivers
	57% m receivers
Calculated Percentages for Experiment 3:
	38.83% f receivers
	61.17% m receivers


## Experiment 4 - handlers get 9/10 of passes
Let's take experiment 3 and take it to a more extreme level and pretend that handlers get 90% of passes. Again we need to construct a new passing function.

In [17]:
def exp4_passing_function(ID, gender, position, the_team, *args):
    hIDs = []
    cIDs = []
    for player in the_team.players:
        if player.position == 'c' and player.ID != ID: cIDs.append(player.ID)
        if player.position == 'h' and player.ID != ID: hIDs.append(player.ID)
    if rand.rand() < 0.9:
        return rand.choice(hIDs)
    else:
        return rand.choice(cIDs)

In [18]:
in_gens = ["m","f","m","f","m","f","m"]
in_poss = ["h","h","h","c","c","c","c"]
pfs = [exp4_passing_function for i in range(7)]
TeamExp4 = Team(in_gens, in_poss, pfs)
SimExp4 = Usage(TeamExp4)
N_throws = 1000000
IDs, gens, poss = SimExp4.simulate(N_throws)

Simulating with 1000000 throws.
Posession starting with:
Player ID 0:
	Gen: m
	Pos: h
	Passing function: exp4_passing_function
Simulation complete with 1000000 throws.


In [19]:
# Analysis on experiment four. 
# Basic statistic - the % of a male receiver vs. the % of a female receiver.
N_possessions = N_throws+1 #One extra for the player starting with the disc.
N_male = gens.count("m")
N_female = gens.count("f")
P_male = float(N_male)/N_possessions*100
P_female = float(N_female)/N_possessions*100
print "Expected Percentages of pure random throws:\n\t43% f receivers\n\t57% m receivers"
print "Calculated Percentages for Experiment 4:\n\t%.2f%% f receivers\n\t%.2f%% m receivers"%(P_female, P_male)

Expected Percentages of pure random throws:
	43% f receivers
	57% m receivers
Calculated Percentages for Experiment 4:
	35.04% f receivers
	64.96% m receivers


## Experiment 5 - dominant male handler
Let's run a new experiment that considers a scenario proposed to create highly imbalanced usage statistic across genders. That is - if one male handler is a "dominant" player and gets the disc very frequently. In this scenario we will write a passing function that actually uses player IDs. The algorithm will be:

* If player 0 (the dominant male) has the disc, pass it to anyone.
* If you are anyone else, you have a 50% chance to pass to player 0 and a 50% chance to pass to anyone else randomly.

In [20]:
def exp5_passing_function(ID, gender, position, the_team, *args):
    if ID == 0:
        return rand.randint(1,7)
    else:
        if rand.rand() < 0.5: 
            return 0
        else:
            N = the_team.N_players
            IDs = range(1,ID)+range(ID+1,N)
            return rand.choice(IDs)

In [21]:
in_gens = ["m","f","m","f","m","f","m"]
in_poss = ["h","h","h","c","c","c","c"]
pfs = [exp5_passing_function for i in range(7)]
TeamExp5 = Team(in_gens, in_poss, pfs)
SimExp5 = Usage(TeamExp5)
N_throws = 1000000
IDs, gens, poss = SimExp5.simulate(N_throws)

Simulating with 1000000 throws.
Posession starting with:
Player ID 0:
	Gen: m
	Pos: h
	Passing function: exp5_passing_function
Simulation complete with 1000000 throws.


In [22]:
# Analysis on experiment five. 
# Basic statistic - the % of a male receiver vs. the % of a female receiver.
N_possessions = N_throws+1 #One extra for the player starting with the disc.
N_male = gens.count("m")
N_female = gens.count("f")
P_male = float(N_male)/N_possessions*100
P_female = float(N_female)/N_possessions*100
print "Expected Percentages of pure random throws:\n\t43% f receivers\n\t57% m receivers"
print "Calculated Percentages for Experiment 5:\n\t%.2f%% f receivers\n\t%.2f%% m receivers"%(P_female, P_male)

Expected Percentages of pure random throws:
	43% f receivers
	57% m receivers
Calculated Percentages for Experiment 5:
	33.39% f receivers
	66.61% m receivers


# Conclusions

All conclusions were reached by running simulations with one million throws. Teams were always composed of three handlers and four cutters, with **two male handlers and one female handler**, and the other four players (two female and two male) are cutters.

Experiment1: 
* If throws were purely random and positions were meaningless, female receivers (or, really, the gender with 3 players) would receive 43% of passes, while male receivers (or the gender with 4 players) would receive 57% of passes.

Experiment 2:

* A assuming two player positions (cutters and handlers) and cutters always threw to handlers and vice versa, females are expected to receive ~42% of passes to ~58% for males. This is a **negligable change from Experiment 1, or a purely random team**.

Experiment 3:

* Assumming that handlers always received 2/3 of passes, regardless of who the thrower was, females are expected to receive ~39% of passes and males ~61% of passes.

Experiment 4:

* Assuming an **extreme scenario where handlers always received 90% of passes**, regardless of who the thrower was, females are expected to receive ~35% of passes and males ~65%.

Experiment 5:

* Assuming that a team has a dominant male handler that always receives half the throws from their teammates, along with one other male handler and one female handler, the females are expected to receive ~33% of passes and males ~67%. If the dominant male handler gets every other throw (so, is thrown to every time by his teammates), then we recover 25% usage statistics for females and 75% for males overall.

In summary, assuming that the majority of teams play 3:4 female-male with two male and one female handlers, it is likely that the results observed in the ultiworld article are a measurement of a throwing bias. That is, in the context of the five experiments shown here, the article **likely did observe a trend where players tended to not throw to female players**.

Alternative explainations that would involve no throwing bias might include:
* Teams ran lots of all male handler lines, possibly suggesting intrinsic, tactical reasons to play males at handler positions in mixed games (possible, but unlikely).
* The players observed in the article have vast skill differences between genders (again, possible, but unlikely given that this is nationals).
* The plays or strategies that teams ran under(over) utilized the female(male) players, meaning there is a 'strategic bias' rather than a throwing bias.
* Mixed play in general allows for defensive strategies that makes it more difficult for females to get open. For instnace, help defense may be easier to execute against female cutters. This is somewhat likely, but is difficult to test in a simulation, and would be much easier to investigate by looking at tape.